In [1]:
using DelimitedFiles
using PyCall
using PyPlot
using Random
using Setfield
using Statistics
using ProgressMeter
using JLD2
using HDF5
using FileIO
using DataStructures
using CUDA
using DrWatson

using Revise

using CMBLensing
using PtsrcLens

┌ Info: Precompiling PtsrcLens [44e3c0cc-451b-45a6-9d84-ed2affe90872]
└ @ Base loading.jl:1278


# Parallel setup

In [2]:
using Distributed, ClusterManagers
cluster_cookie("marius")
em = ElasticManager(addr=:auto, port=9312);

In [5]:
em

ElasticManager:
  Active workers : [ 2,3,4,5,6,7,8,9]
  Number of workers to be added  : 0
  Terminated workers : []
  Worker connect command : 
    /global/home/users/mariusmillea/src/julia-1.5.2/bin/julia --project=/global/home/users/mariusmillea/work/ptsrclens/Project.toml -e 'using ClusterManagers; ClusterManagers.elastic_worker("marius          ","10.0.0.24",9312)'

In [6]:
@everywhere using CUDA, CMBLensing, PtsrcLens

In [7]:
CMBLensing.assign_GPU_workers()

In [8]:
for i in workers()
    @fetchfrom i println(gethostname(),"::",device())
end

      From worker 2:	n0017.savio2::CuDevice(0)
      From worker 3:	n0017.savio2::CuDevice(1)
      From worker 4:	n0017.savio2::CuDevice(2)
      From worker 5:	n0017.savio2::CuDevice(3)
      From worker 6:	n0018.savio2::CuDevice(0)
      From worker 7:	n0018.savio2::CuDevice(1)
      From worker 8:	n0018.savio2::CuDevice(2)
      From worker 9:	n0018.savio2::CuDevice(3)


# Load files

In [45]:
@unpack (ϕs, κs, gs_ir, gs_radio, Ms_radio) = load_hdf5_cutouts();

Progress: 100%|█████████████████████████████████████████| Time: 0:00:19
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:01:32
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
Progress: 100%|█████████████████████████████████████████| Time: 

In [179]:
@save "data/sehgal_maps_h5/cutouts.jld2" ϕs κs gs_ir gs_radio Ms_radio;

## Load pre-processed

In [46]:
@load "data/sehgal_maps_h5/cutouts.jld2"

5-element Array{Symbol,1}:
 :ϕs
 :κs
 :gs_ir
 :gs_radio
 :Ms_radio

# Check power

In [47]:
@unpack (μKarcmin_gs_radio, μKarcmin_gs_ir) = get_foreground_whitenoise_level(;Ms_radio, gs_radio, gs_ir);

In [48]:
μKarcmin_gs_radio

OrderedDict{Tuple{Symbol,Int64,Real},Union{Nothing, Float64}} with 16 entries:
  (:deep, 90, 2)    => nothing
  (:deep, 90, 5)    => nothing
  (:deep, 90, 10)   => 0.183897
  (:deep, 90, Inf)  => 1.46472
  (:deep, 148, 2)   => 0.0346629
  (:deep, 148, 5)   => 0.0579392
  (:deep, 148, 10)  => 0.080185
  (:deep, 148, Inf) => 0.632798
  (:wide, 90, 2)    => 0.0752406
  (:wide, 90, 5)    => 0.131259
  (:wide, 90, 10)   => 0.185232
  (:wide, 90, Inf)  => 1.46472
  (:wide, 148, 2)   => nothing
  (:wide, 148, 5)   => nothing
  (:wide, 148, 10)  => 0.0824392
  (:wide, 148, Inf) => 0.632798

In [49]:
tab = pyimport("pandas").DataFrame(Dict(
    148 => Dict(fluxcut => μKarcmin_gs_radio[:deep,148,fluxcut] for fluxcut in [Inf,10,5,2]),
    90  => Dict(fluxcut => μKarcmin_gs_radio[:wide,90, fluxcut] for fluxcut in [Inf,10,5,2]),
)).T
println(tab.to_latex(float_format="%.2f"))
tab

\begin{tabular}{lrrrr}
\toprule
{} &  inf  &  10.0 &  2.0  &  5.0  \\
\midrule
148 &  0.63 &  0.08 &  0.03 &  0.06 \\
90  &  1.46 &  0.19 &  0.08 &  0.13 \\
\bottomrule
\end{tabular}



,inf,10.0,2.0,5.0
148,0.632798,0.080185,0.034663,0.057939
90,1.464721,0.185232,0.075241,0.131259


In [50]:
μKarcmin_gs_ir

Dict{Int64,Float64} with 1 entry:
  148 => 0.0740397

# Reconstruct

In [10]:
ℓedges = [2:100:500; round.(Int, 10 .^ range(log10(502), log10(6000), length=10))];

In [11]:
noises = Dict(
    (:deep, 90)  => (μKarcminT=0.68/√2, beamFWHM=2.3, ℓknee=200, αknee=2),
    (:wide, 90)  => (μKarcminT=2.9/√2,  beamFWHM=2.2, ℓknee=700, αknee=1.4),
    (:deep, 148) => (μKarcminT=0.96/√2, beamFWHM=1.5, ℓknee=200, αknee=2),
    (:wide, 148) => (μKarcminT=2.8/√2,  beamFWHM=1.4, ℓknee=700, αknee=1.4),
);

In [9]:
ℓmax_datas = [3000, 5000]
polfrac_scales = [1]
freqs = [90, 148]
surveys = [:deep, :wide];

In [ ]:
PtsrcLens.main_MAP_grid(;surveys,freqs,ℓmax_datas,fluxcuts=[5],polfrac_scales=[1])